## Capstone Project Week 3 - Segmenting and Clustering Neighborhoods in Toronto (ALL Part 1,2 and 3)

## 1) Web scraping using BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [2]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
src = result.content
soup = BeautifulSoup(src, 'html.parser')

### grab the first table - postal code table

In [3]:
#print(soup.prettify())
postalcode_html = soup.find_all('table')[0]

### Put html table into dataframe

In [4]:
table_rows = postalcode_html.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
postalcode_table = pd.DataFrame(l, columns=column_names)

### Remove rows that have no proper values in Borough

In [5]:
# Drop any rows with no values in Borough
postalcode_table.dropna(subset=['Borough'], axis=0, inplace=True)    

# Get indexes for which Borough column has 'Not assigned', then delete rows from dataframe
indexNoBorough = postalcode_table[ postalcode_table['Borough'] == 'Not assigned' ].index
postalcode_table.drop(indexNoBorough , inplace=True)

# reset index after dropped rows
postalcode_table.reset_index(drop=True, inplace=True)
postalcode_table

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n
5,M6A,North York,Lawrence Manor\n
6,M7A,Queen's Park,Not assigned\n
7,M9A,Etobicoke,Islington Avenue\n
8,M1B,Scarborough,Rouge\n
9,M1B,Scarborough,Malvern\n


### Clean up Neighborhood values and modify 'Not assigned' Neighborhood with Borough values

In [6]:
# Remove column Neighorhood '\n' carriage return
postalcode_table['Neighborhood'] = postalcode_table['Neighborhood'].str.replace('\n','')

# Neighborhood cell set to same as borough for Neighborhood with value 'Not assigned'
postalcode_table.loc[postalcode_table['Neighborhood'] == 'Not assigned', 'Neighborhood'] = postalcode_table['Borough']
postalcode_table.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Combine Neighborhood based on same PostalCode and Borough

In [7]:
# Combine Neighborhood based on same Postalcode and Borough
postalcode_table = postalcode_table.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ','.join(x))
postalcode_table

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [8]:
postalcode_table.shape

(103, 3)

## 2) Read postal .csv file by the url using pandas and assign it to df

In [9]:
csv_path = "http://cocl.us/Geospatial_data"
df = pd.read_csv(csv_path)
df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df.shape

(103, 3)

## join both dataframes

In [11]:
postalcd_Lat_Lng_table = postalcode_table.join(df, how='outer')
postalcd_Lat_Lng_table.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## remove extra 'Postal Code' column

In [12]:
del postalcd_Lat_Lng_table['Postal Code']
postalcd_Lat_Lng_table

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [13]:
postalcd_Lat_Lng_table.shape

(103, 5)

## 3) Explore and cluster the neighborhoods in Toronto

### Explore and cluster the neighborhood within North York area within Toronto

#### Create a new dataframe of the North York area

In [14]:
northYork_data = postalcd_Lat_Lng_table[postalcd_Lat_Lng_table['Borough'] == 'North York'].reset_index(drop=True)
northYork_data.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(northYork_data['Borough'].unique()),
        northYork_data.shape[0]
    )
)

The dataframe has 1 boroughs and 24 neighborhoods.


#### Use geopy library to get the latitude and longitude values of City North York in Toronto, Ontario.

In [15]:
address = 'North York, ONT'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.5909589, -79.6461044.


#### Create a map of North York, Toronto, Ontario with neighborhoods superimposed on top.

In [17]:
# create map of New York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(northYork_data['Latitude'], northYork_data['Longitude'], northYork_data['Borough'], northYork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

#### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'EIXAIKKXGND2YFYGL2YCMFTFHXC1BCUXULBNCPMCW3PR2NEP' # your Foursquare ID
CLIENT_SECRET = 'E55GTLP2MO4KGK3CLZ3GL2XZ21DUUZUQXI32ODPZPUC2PDOH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EIXAIKKXGND2YFYGL2YCMFTFHXC1BCUXULBNCPMCW3PR2NEP
CLIENT_SECRET:E55GTLP2MO4KGK3CLZ3GL2XZ21DUUZUQXI32ODPZPUC2PDOH


#### Define function to extract category of the venue

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Define function to explore all neighborhood in North York city

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Set up Foursquare LIMIT and radius

In [24]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


#### Write code to run the above function on each neighborhood and create a new dataframe called *northYork_venues*.

In [25]:
northYork_venues = getNearbyVenues(names=northYork_data['Neighborhood'],
                                   latitudes=northYork_data['Latitude'],
                                   longitudes=northYork_data['Longitude']
                                  )

Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Downsview,North Park,Upwood Park
Humber Summit
Emery,Humberlea


#### Let's check the size of the resulting dataframe

In [26]:
print(northYork_venues.shape)
northYork_venues.head()

(251, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


#### Find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(northYork_venues['Venue Category'].unique())))

There are 111 uniques categories.


#### Analyze each neighborhoood

In [29]:
# one hot encoding
northYork_onehot = pd.get_dummies(northYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northYork_onehot['Neighborhood'] = northYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northYork_onehot.columns[-1]] + list(northYork_onehot.columns[:-1])
northYork_onehot = northYork_onehot[fixed_columns]

northYork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview,Henry Farm,Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
northYork_grouped = northYork_onehot.groupby('Neighborhood').mean().reset_index()
northYork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.0,0.0,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.055556,0.000000,0.000000,0.00,0.0000,0.055556,0.055556,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.00,0.00,0.00,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.0000,0.000000,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.04,0.0,0.0,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.080000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.00,0.04,0.040000,0.000000,0.0,0.00,0.00,0.000000,0.040000,0.04,0.000000,0.00,0.080000,0.

#### Find out the new size

In [32]:
northYork_grouped.shape

(23, 112)

#### Print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in northYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northYork_grouped[northYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1                Diner  0.06
2        Shopping Mall  0.06
3           Restaurant  0.06
4  Fried Chicken Joint  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1    Italian Restaurant  0.08
2           Coffee Shop  0.08
3              Pharmacy  0.04
4        Sandwich Place  0.04


----CFB Toronto,Downsview East----
               venue  freq
0            Airport  0.33
1               Park  0.33
2  Other Repair Shop  0.33
3                Gym  0.00
4          Pet Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3   

#### Write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northYork_grouped['Neighborhood']

for ind in np.arange(northYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Grocery Store,Indian Restaurant,Liquor Store,Cupcake Shop,Café,Pharmacy,Pizza Place
3,"CFB Toronto,Downsview East",Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


#### Run k-means to cluster the neighborhood into 3 clusters.

In [50]:
# set number of clusters
kclusters = 4

northYork_grouped_clustering = northYork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 1,
       0])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
northYork_merged = northYork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northYork_merged = northYork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northYork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,3.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
northYork_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,3.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop,Plaza
6,M2P,North York,York Mills West,43.752758,-79.400049,0.0,Park,Convenience Store,Bank,Bar,Electronics Store,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
7,M2R,North York,Willowdale West,43.782736,-79.442259,1.0,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Pharmacy,Asian Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop
8,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
9,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


#### Drop from dataframe on NaN value rows which has no venue ratings

In [59]:
northYork_merged.dropna(subset=['Cluster Labels'], axis=0, inplace=True)

#### Visualize the resulting clusters

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northYork_merged['Latitude'], northYork_merged['Longitude'], northYork_merged['Neighborhood'], northYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1 - Most Green cluster with lots of park

In [63]:
northYork_merged.loc[northYork_merged['Cluster Labels'] == 0, northYork_merged.columns[[1] + list(range(5, northYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,0.0,Park,Convenience Store,Bank,Bar,Electronics Store,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
8,North York,0.0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
13,North York,0.0,Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
21,North York,0.0,Park,Construction & Landscaping,Deli / Bodega,Bakery,Basketball Court,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store


#### Cluster 2 - Most convenient cluster with lots of shops 

In [64]:
northYork_merged.loc[northYork_merged['Cluster Labels'] == 1, northYork_merged.columns[[1] + list(range(5, northYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,North York,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,North York,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
5,North York,1.0,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop,Plaza
7,North York,1.0,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Pharmacy,Asian Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop
9,North York,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
10,North York,1.0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Smoke Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store
11,North York,1.0,Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
12,North York,1.0,Coffee Shop,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Falafel Restaurant,Discount Store,Cosmetics Shop,Cupcake Shop
14,North York,1.0,Grocery Store,Bank,Shopping Mall,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store


#### Cluster 3 - Cluster with special baseball field and stores

In [65]:
northYork_merged.loc[northYork_merged['Cluster Labels'] == 2, northYork_merged.columns[[1] + list(range(5, northYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,2.0,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


#### Cluster 4 - Cluster with many cafeteria

In [66]:
northYork_merged.loc[northYork_merged['Cluster Labels'] == 3, northYork_merged.columns[[1] + list(range(5, northYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,3.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
